In [1]:
import os
import cv2
import keras
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
dirs = 'Train_model/'

In [3]:
imagePaths = [dirs+f 
             for f in os.listdir(dirs)
             if os.path.isfile(os.path.join(dirs,f))]

In [4]:
len(imagePaths)

20858

In [5]:
labels=[]
rawImages=[]

In [6]:
for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath)
    label = imagePath.split('/')[1].split('(')[0]
    image = cv2.resize(image, (28,28))
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #images = np.append(images, [image], axis=0)
    labels.append(label)
    rawImages.append(image)
    
    if i > 0 and i % 1000 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))

[INFO] processed 1000/20858
[INFO] processed 2000/20858
[INFO] processed 3000/20858
[INFO] processed 4000/20858
[INFO] processed 5000/20858
[INFO] processed 6000/20858
[INFO] processed 7000/20858
[INFO] processed 8000/20858
[INFO] processed 9000/20858
[INFO] processed 10000/20858
[INFO] processed 11000/20858
[INFO] processed 12000/20858
[INFO] processed 13000/20858
[INFO] processed 14000/20858
[INFO] processed 15000/20858
[INFO] processed 16000/20858
[INFO] processed 17000/20858
[INFO] processed 18000/20858
[INFO] processed 19000/20858
[INFO] processed 20000/20858


In [7]:
images_array = np.array(rawImages)
images_array.shape


(20858, 28, 28)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(images_array, np.array(labels), test_size=0.2)

In [9]:
unique , counts = np.unique(y_train,return_counts = True)
print('Train data : ',dict(zip(unique,counts)))

Train data :  {'0': 778, '1': 537, '10': 781, '11': 852, '12': 874, '13': 941, '14': 853, '15': 764, '16': 752, '17': 942, '18': 762, '19': 955, '2': 1113, '3': 1028, '4': 895, '5': 777, '6': 713, '7': 776, '8': 920, '9': 673}


In [10]:
unique , counts = np.unique(y_test,return_counts = True)
print('Test data : ',dict(zip(unique,counts)))

Test data :  {'0': 179, '1': 133, '10': 168, '11': 214, '12': 205, '13': 235, '14': 235, '15': 182, '16': 215, '17': 213, '18': 188, '19': 250, '2': 307, '3': 244, '4': 249, '5': 221, '6': 185, '7': 184, '8': 212, '9': 153}


In [11]:
num_label = len(np.unique(y_train)) #10 labels
print(num_label)

20


In [12]:
print(X_train.shape)
print(X_test.shape)

(16686, 28, 28)
(4172, 28, 28)


In [13]:
X_train = X_train.reshape(X_train.shape[0],28,28,1).astype('float32')/255
X_test = X_test.reshape(X_test.shape[0],28,28,1).astype('float32')/255

In [14]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout,Flatten,MaxPooling2D,Conv2D
from keras.utils import to_categorical,plot_model

In [15]:
n_class = 20
y_train = to_categorical(y_train,n_class)
y_test = to_categorical(y_test,n_class)

In [16]:
#model MLP 3 layer
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation = 'relu',input_shape = (28,28,1)))

model.add(Conv2D(64,kernel_size=(3,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense((128),activation='relu'))
model.add(Dropout((0.5)))

model.add(Dense(n_class,activation='softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

In [18]:
model.fit(X_train,y_train , epochs = 15 , batch_size =128)
score =model.evaluate(X_test,y_test ,batch_size = 128)
print("ACCURANCY : " + str(score[1]*100))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/15





16686/16686 [==============================] - 14s 830us/step - loss: 1.0807 - acc: 0.7078
Epoch 2/15
16686/16686 [==============================] - 13s 778us/step - loss: 0.1791 - acc: 0.9536
Epoch 3/15
16686/16686 [==============================] - 13s 784us/step - loss: 0.1099 - acc: 0.9710
Epoch 4/15
16686/16686 [==============================] - 13s 802us/step - loss: 0.0794 - acc: 0.9775
Epoch 5/15
16686/16686 [==============================] - 13s 800us/step - loss: 0.0621 - acc: 0.9815
Epoch 6/15
16686/16686 [==============================] - 13s 764us/step - loss: 0.0509 - acc: 0.9853
Epoch 7/15
16686/16686 [==============================] - 13s 770us/step - loss: 0.0418 - acc: 0.9888
Epoch 8/15
16686/16686 [==============================] - 13s 758us/step - loss: 0.0389 - acc: 0.9881
Epoch 9/15
16686/16686 [==============================] - 13s 787us/step - loss: 0

In [19]:
value = model.predict(X_test)
y_pred =np.argmax(value,axis=1)
y_true = np.argmax(y_test,axis=1)

In [20]:
from sklearn.metrics import classification_report

In [21]:
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99       179
           1       1.00      0.98      0.99       133
           2       1.00      0.99      1.00       307
           3       1.00      1.00      1.00       244
           4       0.99      1.00      0.99       249
           5       0.97      0.98      0.98       221
           6       0.98      0.98      0.98       185
           7       1.00      0.99      1.00       184
           8       0.99      0.99      0.99       212
           9       0.97      0.99      0.98       153
          10       1.00      1.00      1.00       168
          11       1.00      1.00      1.00       214
          12       1.00      1.00      1.00       205
          13       1.00      1.00      1.00       235
          14       1.00      1.00      1.00       235
          15       1.00      1.00      1.00       182
          16       0.99      1.00      1.00       215
          17       1.00    

In [22]:
def class_number(num):
    if num < 10 : return num
    elif num == 10 : return 0
    elif num == 11 : return 1
    elif num == 12 : return 2
    elif num == 13 : return 3
    elif num == 14 : return 4
    elif num == 15 : return 5
    elif num == 16 : return 6
    elif num == 17 : return 7
    elif num == 18 : return 8
    else  : return 9

In [23]:
model.save('model_tensorflow2.h5')